# Overview of py4cytoscape

by Kozo Nishida, Alexander Pico, Tanja Muetze, Georgi Kolishovski, Paul Shannon, Barry Demchak

**Package**

py4cytoscape 0.0.8

Cytoscape is a well-known bioinformatics tool for displaying and exploring biological networks. Python is a powerful programming language and environment for statistical and exploratory data analysis. *py4cytoscape* uses CyREST to communicate between **Python** and Cytoscape, allowing any graphs (e.g., igraph or dataframes) to be viewed, explored and manipulated with the Cytoscape point-and-click visual interface. Thus, via py4cytoscape, these two quite different, quite useful bioinformatics software environments are connected, mutually enhancing each other, providing new possibilities for exploring biological data.

## Installation


In [ ]:
%%capture
!python3 -m pip install python-igraph requests pandas networkx
!python3 -m pip install py4cytoscape

## Prerequisites

In addition to this package (py4cytoscape), you will need:

Cytoscape 3.8.1 or greater + [FileTransfer App](https://apps.cytoscape.org/apps/filetransfer), which can be downloaded from http://www.cytoscape.org/download.php. Simply follow the installation instructions on screen.

Once you have Cytoscape installed in your desktop environment,
launch Cytoscape and keep it running whenever using py4cytoscape.

In addition to that, if you are now running this Notebook on Google Colab, you need to run the code below.
(If you're running this notebook on your local Jupyter, you don't need to.)

In [ ]:
import IPython
import py4cytoscape as p4c
print(f'Loading Javascript client ... {p4c.get_browser_client_channel()} on {p4c.get_jupyter_bridge_url()}')
browser_client_js = p4c.get_browser_client_js()
IPython.display.Javascript(browser_client_js) # Start browser client

## Getting started

First, confirm that you have everything installed and running:

In [ ]:
p4c.cytoscape_ping()

In [ ]:
p4c.cytoscape_version_info()

## My first network

Let’s create a Cytoscape network from some basic Python objects


In [ ]:
import pandas as pd
nodes = pd.DataFrame(data={'id': ["node 0","node 1","node 2","node 3"], 'group': ["A","A","B","B"], 'score': [20,10,15,5]})
edges = pd.DataFrame(data={'source': ["node 0","node 0","node 0","node 2"], 'target': ["node 1","node 2","node 3","node 3"], 'interaction': ["inhibits","interacts","activates","interacts"], 'weight': [5.1,3.0,5.2,9.9]})

p4c.create_network_from_data_frames(nodes, edges, title="my first network", collection="DataFrame Example")

### Switch styles

Check out the marquee style!

In [ ]:
p4c.set_visual_style('Marquee')

### My own style

Create your own style with node attribute fill mappings and some defaults

In [ ]:
style_name = "myStyle"
defaults = {'NODE_SHAPE': "diamond", 'NODE_SIZE': 30, 'EDGE_TRANSPARENCY': 120, 'NODE_LABEL_POSITION': "W,E,c,0.00,0.00"}
nodeLabels = p4c.map_visual_property('node label','id','p')
nodeFills = p4c.map_visual_property('node fill color','group','d',["A","B"], ["#FF9900","#66AAAA"])
arrowShapes = p4c.map_visual_property('Edge Target Arrow Shape','interaction','d',["activates","inhibits","interacts"],["Arrow","T","None"])
edgeWidth = p4c.map_visual_property('edge width','weight','p')

p4c.create_visual_style(style_name, defaults, [nodeLabels,nodeFills,arrowShapes,edgeWidth])
p4c.set_visual_style(style_name)

*Pro-tip: if you want to set NODE_WIDTH and NODE_HEIGHT independently, you also need to unlock the node dimensions with…*

In [ ]:
p4c.lock_node_dimensions(False, style_name)

## Biological graph example

Here we create a 4-node graph in Python, send it to Cytoscape for display and layout. For the sake of simplicity, no node attributes and no visual styles are included; those topics are covered in subsequent steps.

In [ ]:
nodes = pd.DataFrame(data={'id': ["A", "B", "C", "D"]})
edges = pd.DataFrame(data={'source': ["C", "B", "B", "B"], 'target': ["D", "A", "D", "C"]})

p4c.create_network_from_data_frames(nodes, edges, title="simple network", collection="Biological Example")

You should now have the structure of this 4-node graph with a basic, default style. Fortunately, Cytoscape has some built-in rendering rules in which (and unless instructed otherwise) nodes and edges are rendered and a default (user-preference) layout algorithm is applied.

### Add node attributes

We often know quite a lot about the nodes and edges in our graphs. By conveying this information visually, the graph will be easier to explore. For instance, we may know that protein A phosphorylates protein B, that A is a kinase and B a transcription factor, and that their mRNA expression (compared to a control) is a log2 fold change of 1.8 and 3.2 respectively. One of the core features of Cytoscape is visual styles, which allow you to specify how data values (e.g., kinase,transcription factor; expression ratios) should be conveyed in the visual properties of the graph (e.g., node shape, node color or size).

We continue with the simple 4-node graph, adding two kinds data values (moleculeType and log2fc). The easiest way to do this is via pandas DataFrame s. However, you can also include attributes together with the original graph models as igraph s or pandas DataFrame s and then use the provided create functions to create and load in a single step (see *p4c.create_network_from_igraph() and p4c.create_network_from_data_frames()* functions). Check out the other Notebooks for more examples.


In [ ]:
df = pd.DataFrame(data={'moleculeType': ['kinase','TF','cytokine','cytokine'], 'log2fc': [1.8,3.0,-1.2,-2.5]})
df.index = ['A','B','C','D']

In [ ]:
p4c.load_table_data(df)

Note that adding the attributes does not in itself cause the appearance of the graph to change. Such a change requires that you specify and apply visual style mappings, which will be explained in the *next* section. You can, however, examine these attributes in Cytoscape, using Cytoscape’s the **Data Panel** to display data values associated with selected nodes immediately below the Cytoscape window.

### Modifying the display: defaults and mappings

py4cytoscape provides an easy way to not only change the default styles, but more interestingly, py4cytoscape also provides easy access to *mapping* your data to visual styles, e.g., allowing the size, shape and color of nodes and edges to be determined by the data you have associated with those nodes and edges.

First, let’s change the the defaults.

In [ ]:
p4c.set_node_shape_default('OCTAGON')

In [ ]:
p4c.set_node_color_default('#AAFF88')

In [ ]:
p4c.set_node_size_default(60)

In [ ]:
p4c.set_node_font_size_default(30)

Now we will add some visual mappings. Let’s map `moleculeType’ to node shapes. First, we can see which shapes are available in Cytoscape, then we can define the mapping with paired lists.


In [ ]:
p4c.get_node_shapes()

In [ ]:
column = 'moleculeType'
values = ['kinase', 'TF', 'cytokine']
shapes = ['DIAMOND', 'TRIANGLE', 'RECTANGLE']

p4c.set_node_shape_mapping(column, values, shapes)

The node shape mapping is an example of a *discrete* mapping, where a style is defined for each, discrete value. This is useful for categorical data (like type) where there is only a limited set of possible values. This is in contast to the other two other types of mappings: *continuous* and *passthrough*. In the case of expression values, for example, we will want to use *continuous* mapping (e.g., to node color), defining a small set of control points, rather than an explicit color for each possible data value. Cytoscape will simply interpolate between the control points to provide a gradient of colors. Let’s try that one now

In [ ]:
column = 'log2fc'
control_points = [-3.0, 0.0, 3.0]
colors = ['#5588DD', '#FFFFFF', '#DD8855']

p4c.set_node_color_mapping(column, control_points, colors)

Note that there are three colors and three control points. However, you can also specify *two additional* colors beyond the number of control points if you want to set extreme (or out-of-bounds) colors for values less than or greater than your control points.

In [ ]:
control_points = [-2.0, 0.0, 2.0]
colors = ['#2255CC', '#5588DD', '#FFFFFF', '#DD8855', '#CC5522']

p4c.set_node_color_mapping(column, control_points, colors)

Now, add a node size rule, using *log2fc* again as controlling node values.

In [ ]:
control_points = [-3.0, 2.0, 3.0]
sizes = [20, 80, 90]
p4c.set_node_size_mapping(column, control_points, sizes)

If you recall the third type of mapping, *passthrough*, we can see it already working in our current network example. The node labels! By default, the *name* column is mapped to the node label property using *passthrough* logic: the value is passed directly to the style property.

### Selecting nodes

Let us now try selecting nodes in Cytoscape from Python. Select the C node by name:


In [ ]:
p4c.select_nodes('C', 'name')

In [ ]:
p4c.get_selected_nodes()

Now we wish to extend the selected nodes to include the first neighbors of the already-selected node B. This is a common operation: for instance, after selecting one or more nodes based on experimental data or annotation, you may want to explore these in the context of interaction partners (in a protein-protein network) or in relation to upstream and downstream partners in a signaling or metabolic network. Type:

In [ ]:
p4c.select_first_neighbors()

You will see that three nodes are now selected. Get their names back to Python as a list:

In [ ]:
node_names = p4c.get_selected_nodes()

In [ ]:
node_names

And, finally, deselection works as you’d expect by means of a general *p4c.clearSelection()* function:

In [ ]:
p4c.clear_selection()

In [ ]:
?p4c.clear_selection

## Browse available functions, commands and arguments

py4cytoscape functions

In [ ]:
help(p4c)

In [ ]:
dir(p4c)

Category | Description | Examples
--- | --- | ---
apps | Inspecting and managing apps for Cytoscape. | *install_app<br/> disable_app<br/> get_installed_apps*
collections | Getting information about network collections. | *get_collection_list<br/> get_collection_networks*
commands | Constructing any arbitrary CyREST API or Commands API method via standard GET, PUT, POST and DELETE protocols. | *cyrest_get<br/> commands_post<br/> cyrest_api<br/> commands_run*
cy_ndex | Communicating with NDEx from within Cytoscape. | *import_network_from_ndex<br/> export_network_to_ndex<br/>*
cytoscape_system | Checking Cytoscape System information, including versions and memory usage. | *cytoscape_ping<br/> cytoscape_version_info<br/>*
filters |	Selecting nodes and edges based on filter criteria. |*create_degree_filter<br/> create_column_filter<br/>*
groups | Working with groups in Cytoscape. | *create_group<br/> collapse_group<br/>*
layouts | Performing layouts in addition to getting and setting layout properties. | *layout_network<br/> get_layout_names<br/>*
networks | Creating and managing networks and retrieving information on networks, nodes and edges. | *create_network_from…<br/> create…_from_network<br/> get_network_suid<br/> export_network<br/> get_all_nodes<br/> get_edge_count<br/> get_first_neighbors<br/>*
network_selection | Manipulating selection of nodes and edges in networks. | *select_nodes<br/> invert_node_selection<br/> select_first_neighbors<br/>*
network_views | Performing view operations in addition to getting and setting view properties. | *fit_content<br/> export_image<br/> toggle_graphics_details*
session | Managing Cytoscape sessions, including save, open and close. | *open_session<br/> save_session<br/> close_session*
style_bypasses | Setting and clearing bypass values for visual properties. | *set_node_color_bypass<br/> set_edge_line_style_bypass<br/> hide_nodes*
style_defaults | Getting and setting default values for visual properties. | *set_node_shape_default<br/> set_edge_line_width_default*
style_dependencies | Getting and setting style dependencies. | *lock_node_dimensions*
style_mappings | Defining mappings between table column values and visual properties. | *map_visual_property<br/> update_style_mapping<br/> set_node_size_mapping<br/> set_edge_color_mapping*
styles | Managing styles and retrieving general lists of properties relevant to multiple style modes. | *create_visual_style<br/> set_visual_style<br/> export_visual_styles<br/> get_arrow_shapes*
style_values | Retrieving current values for visual properties. | *get_node_width<br/> get_edge_color<br/> get_network_zoom*
tables | Managing table columns and table column functions, like map and rename, as well as loading and extracting table data in Cytoscape. | *get_table_columns<br/> rename_table_column<br/> load_table_data<br/> map_table_column*
tools | Performing actions found in the Tools Menu in Cytoscape. | *cybrowser_dialog<br/> diffusion_basic*
user_interface | Controling the panels in the Cytoscape user interface. | *hide_panel<br/> float_panel<br/> dock_panel*


## More examples

The Cytoscape team is collecting scripts from the community in a public GitHub repository at https://github.com/cytoscape/cytoscape-automation/tree/master/for-scripters/Python.

## Development

The py4cytoscape project code and documentation is maintained at GitHub: https://github.com/cytoscape/py4cytoscape. All bugs and feature requests are tracked as Issues, https://github.com/cytoscape/py4cytoscape/issues.

## Credits

- Paul Shannon’s generous advice and mentorship was very important for transforming this package from using XMLRPC and CytoscapeRPC to using CyREST.
- David Otasek, Keiichiro Ono and Barry Demchak kindly provided CyREST as well as help and support for new functionalities and changes.
- Mark Grimes and Ruth Isserlin kindly provided helpful user feedback.
- Julia Gustavsen generously developed various use cases/examples for using RCy3 with biological data during GSOC 2016, https://github.com/jooolia/gsoc_Rcy3_vignettes/blob/master/blog_post_drafts/final_work_submission.md.
- Tanja Muetze provided many years of development, design, maintenance and documentation for the RCy3 project.
- All contributors, new and old, are dynamically acknowledged in our Contributor Graph, https://github.com/cytoscape/py4cytoscape/graphs/contributors

## References

1. Shannon P, Markiel A, Ozier O, Baliga NS, Wang JT, Ramage D, Amin N, Schwikowski B, Ideker T. 2003. Cytoscape: a software environment for integrated models of biomolecular interaction networks. Genome Res. Nov;13(11):2498-504
2. Huber W, Carey VJ, Long L, Falcon S, Gentleman R. 2007. Graphs in molecular biology. BMC Bioinformatics. 2007 Sep 27;8.
3. Ono K, Muetze T, Kolishovski G, Shannon P, Demchak, B. CyREST: Turbocharging Cytoscape Access for External Tools via a RESTful API [version 1; referees: 2 approved]. F1000Research 2015, 4:478.
